In [1]:
import torch

# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [7]:
# IMPORTS

# Torch Features
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler

# Calc Utilities
import numpy as np
from collections import OrderedDict

# Plot Utilities
import matplotlib.pyplot as plt
%matplotlib inline

# Miscellaneous
import time


In [8]:
# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 20
# percentage of training set to use as validation
valid_size = 0.1

In [10]:
train_data = False #TO DO
test_data = False #TO DO

# obtain training indices that will be used for validation
num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# prepare data loaders (combine dataset and sampler)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
    sampler=train_sampler, num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
    sampler=valid_sampler, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
    num_workers=num_workers)


TypeError: object of type 'bool' has no len()

In [11]:
def train_model(model, checkpoint_path = False):
    
    # Start epoch variable
    start_epoch = 0
    
    # Define the Loss
    criterion = nn.NLLLoss()

    # Only train the classifier parameters, feature parameters are frozen
    optimizer = optim.Adadelta(model.fc.parameters())
    
    #Check if there is an old run
    if checkpoint_path:
      cfile = torch.load(checkpoint_path)
      start_epoch = cfile['epochs']
      optimizer.load_state_dict(cfile['optim'])


    # number of epochs to train the model
    n_epochs = 15 
    
    valid_loss_min = np.Inf # track change in validation loss

    for epoch in range(1, n_epochs + 1):
        start_time = time.time()
      
        # Move model to GPU if CUDA is available
        if train_on_gpu:
            model.cuda()

        # keep track of training and validation loss
        train_loss = 0.0
        valid_loss = 0.0

        ###################
        # train the model #
        ###################
        model.train()
        for data, target in train_loader:
            # move tensors to GPU if CUDA is available
            if train_on_gpu:
                data, target = data.cuda(), target.cuda()
            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # calculate the batch loss
            loss = criterion(output, target)
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            # update training loss
            train_loss += loss.item()*data.size(0)

        ######################    
        # validate the model #
        ######################
        correct = 0
        total = 0
        model.eval()
        for data, target in valid_loader:
            # move tensors to GPU if CUDA is available
            if train_on_gpu:
                data, target = data.cuda(), target.cuda()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # calculate the batch loss
            loss = criterion(output, target)
            # update average validation loss 
            valid_loss += loss.item()*data.size(0)
            
            #calculate accuracy

        # calculate average losses


        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} \ttValidation Acc.: {:.2f} \tDuration: {:.2f} min.'.format(
            start_epoch + epoch, train_loss, valid_loss,accuracy,duration))

        # save model if validation loss has decreased



In [ ]:
if __name__ == '__main__':
    #TO-DO